In [11]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [6]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
# bets.drop([5,6,7], inplace = True)
# bets.reset_index(inplace = True, drop = True)
# bets.to_csv('In_Season/Data/todays_bets.csv')

In [17]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
# results

In [28]:
# Map for 538 scraping
fivethirtyeight_team_map =  {
    'Pistons' : 'Detroit Pistons',
    'Thunder' : 'Oklahoma City Thunder',
    'Kings' : 'Sacramento Kings',
    'Trail Blazers' : 'Portland Trailblazers',
    'Spurs' : 'San Antonio Spurs',
    'Raptors' : 'Toronto Raptors',
    'Rockets' : 'Houston Rockets',
    'Magic' : 'Orlando Magic',
    'Warriors' : 'Golden State Warriors',
    'Celtics' : 'Boston Celtics',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Hornets' : 'Charlotte Hornets',
    'Pacers' : 'Indiana Pacers',
    'Grizzlies' : 'Memphis Grizzlies',
    'Wizards' : 'Washington Wizards',
    'Knicks' : 'New York Knicks',
    'Nets' : 'Brooklyn Nets',
    'Bucks' : 'Milwaukee Bucks',
    'Bulls' : 'Chicago Bulls',
    'Pelicans' : 'New Orleans Pelicans',
    'Jazz' : 'Utah Jazz',
    'Nuggets' : 'Denver Nuggets',
    'Clippers' : 'Los Angeles Clippers',
    'Lakers' : 'Los Angeles Lakers',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Heat' : 'Miami Heat',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Suns' : 'Phoenix Suns'
}

In [19]:
# Kelly criterion functions
def kc(row, Home):
    if Home:
        diff = row.Home_Prob - row.Home_Prob_Implied
        if diff < 0:
            return 0
        else:
            p = row.Home_Prob
            q = 1-p
            ml = row.Home_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            return (kc/10.0)
    else:
        diff_away = row.Away_Prob - row.Away_Prob_Implied
        if diff_away < 0:
            return 0
        else:
            p = row.Away_Prob
            q = 1-p
            ml = row.Away_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            return (kc/10.0)

def kc_combined(row, Home):
    if Home:
        diff = row.Home_Prob_Combined - row.Home_Prob_Implied
        if diff < 0:
            return 0
        else:
            p = row.Home_Prob_Combined
            q = 1-p
            ml = row.Home_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            return (kc/10.0)
    else:
        diff_away = row.Away_Prob_Combined - row.Away_Prob_Implied
        if diff_away < 0:
            return 0
        else:
            p = row.Away_Prob_Combined
            q = 1-p
            ml = row.Away_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            return (kc/10.0)

In [20]:
# Scraping data from 538
tables = pd.read_html(f'https://projects.fivethirtyeight.com/{current_year}-nba-predictions/games/')
fivethirtyeight_data = pd.DataFrame(columns = ['Date', 'Away_Team', 'Away_Prob', 'Home_Team', 'Home_Prob'])
for i, table in enumerate(tables):
    if i == 0:
        continue
    if i > 30:
        break
    if i % 2 == 0:
        matchup_data = table.iloc[:, [2,4]]
        matchup_data.drop(2, axis = 0, inplace = True)
        matchup_data.columns = ['Teams', 'Probability']
        away_team = matchup_data.loc[0, 'Teams']
        away_prob = matchup_data.loc[0, 'Probability']
        home_team = matchup_data.loc[1, 'Teams']
        home_prob = matchup_data.loc[1, 'Probability']
        date = dt.date.today()
        data_series = pd.Series([date, away_team, away_prob, home_team, home_prob], index = fivethirtyeight_data.columns)
        fivethirtyeight_data = fivethirtyeight_data.append(data_series, ignore_index = True)

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
# Retreiving odds


# Creating function to convert odds to probability
def calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

# Creating team map to align w/ final output
team_map = {
    'Wizards' : 'Washington Wizards',
    'Celtics' : 'Boston Celtics',
    'Pelicans' : 'New Orleans Pelicans',
    'Knicks' : 'New York Knicks',
    'Pistons' : 'Detroit Pistons',
    'Magic' : 'Orlando Magic',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Pacers' : 'Indiana Pacers',
    'Raptors' : 'Toronto Raptors',
    'Grizzlies' : 'Memphis Grizzlies',
    'Heat' : 'Miami Heat',
    'Bulls' : 'Chicago Bulls',
    'Jazz' : 'Utah Jazz',
    'Bucks' : 'Milwaukee Bucks',
    'Spurs' : 'San Antonio Spurs',
    'Warriors' : 'Golden State Warriors',
    'Thunder' : 'Oklahoma City Thunder',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Nuggets' : 'Denver Nuggets',
    'Suns' : 'Phoenix Suns',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Kings' : 'Sacramento Kings',
    'Hornets' : 'Charlotte Hornets',
    'Trail Blazers' : 'Portland Trailblazers',
    'Nets' : 'Brooklyn Nets',
    'Lakers' : 'Los Angeles Lakers',
    'Clippers' : 'Los Angeles Clippers',
    'Rockets' : 'Houston Rockets'
}

# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nba/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting table from HTML
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds.dropna(inplace = True)
odds.reset_index(drop = True, inplace = True)

odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():

    # Retreiving teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]

    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 12:
        ml_string = ml_string.replace('ML', '')
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 13:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    if len(ml_string) == 14:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[5] == '+') | (ml_string[5]=='-'):
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])
odds = odds_df



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache
